In [7]:
from sklearn.linear_model import LinearRegression
from statsmodels.stats.diagnostic import het_white
import statsmodels.api as sm
import pandas as pd
import numpy as np
import os

In [8]:
df = pd.read_excel('../output/data/gold/filtred_with_coordinates.xlsx')

In [10]:
df_somente_centro = df[df['cep_dest']=="Centro"].drop(columns=['rua/avenida','bairro', 'Zona', 'cep_dest', 'latitude_origin',
                                                               'longitude_origin',	'latitude_destino', 'longitude_destino', 'cidade', 'estado', 'cep', 'cep_origin', 'iptu', 'custo condominio', 'preco'])

df_somente_centro_ordened_columns = df_somente_centro[['quartos', 'banheiros', 'vagas garagem', 'area util por m²', 'distance_between_ceps_in_meters',
                                                       'armários na cozinha','academia', 'piscina', 'varanda', 'portaria', 'quarto de serviço', 'área de serviço', 
                                                       'condomínio fechado', 'elevador', 'permitido animais', 'churrasqueira',  'mobiliado', 'armários no quarto', 
                                                       'ar condicionado', 'segurança 24h', 'salão de festas']]

df_somente_centro_ordened_columns.corr().to_excel('corr_centro.xlsx')

In [11]:
for col in df.columns:
    df[col] = df[col].replace(-1, 0)

df['log_preco'] = np.log(df['preco'])
df['log_area_util'] = np.log(df['area util por m²'])
df['log_distance_ceps'] = np.log(df['distance_between_ceps_in_meters'])

df.drop(columns=['custo condominio','iptu', 'preco', 'area util por m²', 'distance_between_ceps_in_meters',
                 'rua/avenida','cidade', 'estado', 'cep', 'cep_origin', 'latitude_origin', 'longitude_origin',
                 'latitude_destino', 'longitude_destino'], inplace=True)

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

df_p_zonas = df.drop(columns=['bairro'])

output_folder = 'resultado_modelos'
os.makedirs(output_folder, exist_ok=True)

lm = LinearRegression()

df_p_zonas = df_p_zonas[~df_p_zonas['Zona'].isin(['Rural', 'Extremo Norte'])]

for infra in df_p_zonas['cep_dest'].unique():
    df_treated = df_p_zonas.loc[
        df_p_zonas['cep_dest'] == infra
    ].drop(columns=['Zona', 'cep_dest'])
    
    y = df_treated['log_preco']
    X = df_treated.drop(columns=['log_preco'])
    
    if len(df_treated) > 2:
        X = sm.add_constant(X)

        model = sm.OLS(y, X).fit(cov_type='HC3')

        summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
        summary_df = summary_df.round(4)

        white_test = het_white(model.resid, model.model.exog)
        white_test_results = pd.DataFrame({
            'Estatística': [white_test[0]],
            'p-value': [white_test[1]]
        }, index=['White Test']).round(4)
        
        r_squared = model.rsquared
        r_squared_adj = model.rsquared_adj
        r_squared_df = pd.DataFrame({
            'R²': [r_squared],
            'R² Ajustado': [r_squared_adj]
        }, index=['Estatísticas']).round(4)
        
        f_test_stat = model.fvalue
        f_test_p_value = model.f_pvalue
        f_test_results = pd.DataFrame({
            'F-Estatística': [f_test_stat],
            'p-value': [f_test_p_value]
        }, index=['F-Test']).round(4)

        file_name = f"Regressao_{infra}.xlsx"
        output_path = os.path.join(output_folder, file_name)

        with pd.ExcelWriter(output_path) as writer:
            summary_df.to_excel(writer, sheet_name='Regressao_Resultados')
            white_test_results.to_excel(writer, sheet_name='Teste_White')
            r_squared_df.to_excel(writer, sheet_name='R²_Estatisticas')
            f_test_results.to_excel(writer, sheet_name='Teste_F')